# Anomaly Detection - MOG

1. 관측 데이터가 다수의 가우시안 분포로부터 생성되었다고 가정하여 확률 밀도 함수 도출하기 

2. E / M 알고리즘을 통해서 객체 확률 / 가우시안 파라미터들의 값을 각각 최적화하기 

3. 적절한 가우시안 분포 개수 정하기 => class 밖에서 확인. 

**구현해야 하는 것**
- $g(x|\mu_m, \sum_m) = \frac{1}{(2\pi)^{d/2} |\sum_m|^{\frac{1}{2}}}exp(\frac{1}{2}(x-\mu_m)^T\sum_m^{-1}(x-\mu_m))$
- $p(x|\lambda) = \sum_{m=1}^Mw_mg(x|\mu_m, \sum_m)$

- 임의의 파라미터 값들을 부여하기 

- E Step : 가우시안 파라미터를 고정하여 객체 확률 값 계산 
> $p(m|x_i, \lambda) = \frac{w_mg(x_i|\mu_m, \sum_m)}{\sum_{k=1}^M w_mg(x_i|\mu_m, \sum_m)}$

- M Step : 객체 확률 값을 고정하여 가우시안 파라미터 값들 계산하기 

> $w_m^{new} = \frac{1}{N} \sum_{i=1}^N p(m|x_i, \lambda)$ 

> $\mu_m^{new} = \frac{\sum_{i=1}^N p(m|x_i, \lambda)x_i}{\sum_{i=1}^Np(m|x_i, \lambda)}$

> $\sigma_m^{2(new)} = \frac{\sum_{i=1}^N p(m|x_i, \lambda)x_i^2}{\sum_{i=1}^N p(m|x_i,\lambda)} - \mu_m^{2(new)}$



**필요한 것**
- X 
- d 
- m
- $\lambda = {w_m, \mu_m, \sum_M}$ 
- Var_type : Spherical / diagonal / full 
>  이건 이번에 고려 X. 기본적으로 Diagonal을 가정 



**함수의 형태**
- def __init__(self, X, Var_type) : 
  
- def lambda(self) : => m개의 [w_1, $\mu_1, \sum_1$] list를 포괄한 list제작 
> 분산의 형태에 대해서 지정해줄 수 있어야 함. 

- def expectation(self) : self.lambda를 입력값으로 받아 self.p 값 업데이트 
- def maximization(self) : self.p 를 입력값으로 받아 self.lambda 값 업데이트 

- def sol(self) : E/M 과정의 답이 오차 내로 줄어들때까지 진행. 최후 우도값으로 반환할 수 있을 것. 


In [16]:
import numpy as np
import pandas as pd
import random as rand

from sklearn.datasets import load_iris
X = load_iris()['data']

import matplotlib.pyplot as plt
import scipy as sc
from scipy.stats import norm
from sys import maxsize

In [ ]:
print(X['DESCR'])

In [67]:
class MOG() : 
    def __init__(self, X, m) : 
        self.X = X 
        self.n = np.shape(X)[0] 
        self.d = np.shape(X)[1]
        
        self.m = m 
        
        self.w, self.mu, self.sigma, self.cov = self.set_parameter()
        self.p = self.expectation()
        
    def set_parameter(self) : 
        # w 는 m분의 1로 균등하게 부여 
        w = np.ones(self.m) / self.m 
        
        # mu는 각 변수들의 최소 ~ 최대 값 사이의 랜덤한 값으로 부여 
        min_value = np.apply_along_axis(lambda a : np.min(a), 0, self.X)
        max_value = np.apply_along_axis(lambda a : np.max(a), 0, self.X)
        mu = [] 
        sigma = [] 
        for i in range(self.m) : 
            mu.append([np.random.uniform(max_value[j], min_value[j]) for j in range(self.d)])
            sigma.append([np.random.uniform(max_value[j] - mu[i][j], mu[i][j] -min_value[j]) for j in range(self.d)])    
        mu = np.array(mu)
        sigma = np.array(sigma)
        cov = [np.diag(sigma[i]) for i in range(self.m)] 
        
        return w, mu, sigma, cov

    
    def g(self, x, m) : 
        x = np.array(x)
        inv_cov = np.linalg.inv(self.cov[m])
        upper_value =  np.exp((x-self.mu[m])@inv_cov@(x-self.mu[m]).T)
        under_value = ((2*np.pi)**(self.d/2)) * np.sqrt((self.cov[m]**2).sum()) 
        return upper_value / under_value

    def expectation(self): 
        p = np.zeros((self.n, self.m))
        
        for i in range(self.n) : 
            vector = np.array([self.g(self.X[i], j) for j in range(self.m)]) 
            p[i] = vector / vector.sum()
        return p # m x n 행렬 
    
    def maximization(self) :
        w = self.p.sum(axis= 0) / self.n
        mu = 
   
        

In [68]:
test = MOG(X, 3)
test.p



array([[7.13211164e-02, 9.25006366e-01, 3.67251805e-03],
       [2.46660752e-02, 9.72613272e-01, 2.72065325e-03],
       [1.24729454e-02, 9.84842757e-01, 2.68429779e-03],
       [1.15580351e-02, 9.85201040e-01, 3.24092497e-03],
       [4.55760471e-02, 9.50421151e-01, 4.00280207e-03],
       [3.98161552e-01, 5.96650463e-01, 5.18798452e-03],
       [1.07932583e-02, 9.85675557e-01, 3.53118498e-03],
       [4.74639215e-02, 9.48586918e-01, 3.94916041e-03],
       [7.35617729e-03, 9.90136685e-01, 2.50713821e-03],
       [3.03604535e-02, 9.66435122e-01, 3.20442436e-03],
       [3.74857748e-01, 6.21732703e-01, 3.40954878e-03],
       [2.40343403e-02, 9.71455266e-01, 4.51039320e-03],
       [1.84829333e-02, 9.78863212e-01, 2.65385505e-03],
       [5.97232938e-03, 9.92256720e-01, 1.77095027e-03],
       [9.38337095e-01, 6.13495351e-02, 3.13370008e-04],
       [8.74878605e-01, 1.23776477e-01, 1.34491813e-03],
       [3.07491483e-01, 6.88960095e-01, 3.54842138e-03],
       [6.66042570e-02, 9.29628

In [51]:
np.shape(X)

(150, 4)

In [63]:
a = np.array([[1,2]])
b = np.array([[3,4],[4,3]])
c = np.array([[2,3]])

b[1,1]

3

In [61]:
np.sqrt((b**2).sum())

7.0710678118654755